In [12]:
pip install gymnasium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install tqdm

In [14]:
pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 5.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 7.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install torch

In [1]:
import numpy as np
import torch
from env import MultiAgentEnv
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import pandas as pd
import json

In [2]:
X_train, y_train = np.load('data/X_train.npy', allow_pickle=True), np.load('data/y_train.npy', allow_pickle=True)
env = MultiAgentEnv(X_train, y_train)

In [3]:
X_train_df = pd.read_csv('/Users/louisedurand-janin/Documents/GitHub/HrFlow_Data_Challenge/data/X_train.csv')
X_train_df['employee embedding'] = X_train_df['employee embedding'].apply(lambda x: np.array(json.loads(x), dtype=np.float32))
X_train_df['company embedding'] = X_train_df['company embedding'].apply(lambda x: np.array(json.loads(x), dtype=np.float32))

In [4]:
# Convertir les colonnes d'embeddings en tensors PyTorch
employee_embedding_tensor = torch.tensor(np.vstack(X_train_df['employee embedding'].values), dtype=torch.float32)
company_embedding_tensor = torch.tensor(np.vstack(X_train_df['company embedding'].values), dtype=torch.float32)

# Concaténer les deux tensors le long de la dimension appropriée (axis=1 pour ajouter des colonnes)
combined_tensor = torch.cat([employee_embedding_tensor, company_embedding_tensor], dim=1)


In [5]:
combined_tensor.shape

torch.Size([29273, 64])

In [37]:
def collect_samples(env, horizon, disable_tqdm=False, print_done_states=False):
    s, _ = env.reset()
    s = s.reshape(-1, 1)
    print('shapes ', s.shape)
    #dataset = []
    S = []
    A = []
    #R = []
    rewards = []
    S2 = []
    D = []
    SA = torch.empty(0, 66)

    for _ in tqdm(range(horizon), disable=disable_tqdm):
        a = env.sample_action()
        a_reshape =  a.reshape(-1,1)
        print('shape a', a.shape)
        s2, r, done, _ = env.step(a)
        #dataset.append((s,a,r,s2,done,trunc))
        # state = current_position + embedding
        embeddings_concat = torch.cat([combined_tensor, s], dim=1)
        print("emebdding_concat shape", embeddings_concat.shape)
        print("shape s", s.shape)
        S.append(embeddings_concat)
        print('shape S', len(S))
        sa = torch.cat([combined_tensor, s, a_reshape], dim=1)
        SA = torch.cat([SA, sa], dim=0)
        print("SA", SA.shape)
        A.append(a.reshape(-1,1))
        rewards = rewards + r
        print("reward", len(rewards))
        S2.append(s2)
        D.append(done)
        
        for i in range(env.num_agents):
            if done[i]:
                #print('done', i)
                s[i] = 0
                env.state[i] = 0
            else:
                s[i] = s2[i]
                env.state[i] = s2[i]
    print("final shape S :", len(S))
    S = np.array(S)
    SA = np.array(SA)
    print('len A:',len(A))
    A = np.array(A)
    #R = np.array(R)
    #rewards = rewards.numpy()
    S2= np.array(S2)
    D = np.array(D)
    return S, A, rewards, S2, D, SA

In [8]:


def rf_fqi(S, A, R, S2, D, iterations, nb_actions, gamma, disable_tqdm=False):
    nb_samples = S.shape[0]
    Qfunctions = []
    SA = np.append(S,A,axis=1)
    for iter in tqdm(range(iterations), disable=disable_tqdm):
        if iter==0:
            value=R.copy()
        else:
            Q2 = np.zeros((nb_samples,nb_actions))
            for a2 in range(nb_actions):
                A2 = a2*np.ones((S.shape[0],1))
                S2A2 = np.append(S2,A2,axis=1)
                Q2[:,a2] = Qfunctions[-1].predict(S2A2)
            max_Q2 = np.max(Q2,axis=1)
            value = R + gamma*(1-D)*max_Q2
        Q = RandomForestRegressor()
        Q.fit(SA,value)
        Qfunctions.append(Q)
    return Qfunctions


In [40]:
career_env = MultiAgentEnv(combined_tensor, y_train)
S, A, R, S2, D, SA = collect_samples(career_env, 1
)

shapes  torch.Size([29273, 1])


  0%|          | 0/1 [00:00<?, ?it/s]

shape a torch.Size([29273])
emebdding_concat shape torch.Size([29273, 65])
shape s torch.Size([29273, 1])
shape S 1
SA torch.Size([29273, 66])
reward 29273


100%|██████████| 1/1 [00:02<00:00,  2.94s/it]

final shape S : 1
len A: 1


In [ ]:
qfunctions = rf_fqi(S, A, R, S2, D, 5, nb_actions, gamma)

In [100]:
print(len(SA))
print(SA[0].shape)

4
(29273, 66)


In [34]:
print(R)

[[1. 0. 0. ... 0. 1. 1.]
 [1. 1. 0. ... 0. 1. 1.]
 [1. 0. 1. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 1. 1.]]


In [41]:
#Test
from sklearn.ensemble import RandomForestRegressor
import numpy as np

#SA = np.append(S,A,axis=1)
value = R.copy()

Q1 = RandomForestRegressor()
Q1.fit(SA,value)
# compute MSE
print("training MSE:", np.mean((value-Q1.predict(SA))**2))

training MSE: 0.016746380343851182
